# Week 7 — Part 02: Config and Secrets Management Lab

**Estimated time:** 60–90 minutes

---

## Pre-study (Self-learn)

Foundamental Course assumes Self-learn is complete. If you need a refresher on environments and configuration:

- [Foundamental Course Pre-study index](../PRESTUDY.md)
- [Self-learn — Chapter 2: Python and Environment Management](../self_learn/Chapters/2/Chapter2.md)

---

## What success looks like (end of Part 02)

- You can separate config from secrets.
- You can load secrets from environment variables.
- You never hardcode API keys in your code.

### Checkpoint

After running this notebook:
- You have a pattern for loading `.env` files
- You can validate required env vars

## Learning Objectives

- Separate configuration from secrets
- Load secrets from environment variables
- Implement secure secret management

## Overview

Goal: separate reproducible config from secrets.

In this lab you will:

- load `.env` early (if available)
- require env vars with clear errors
- build a small, stable config dict

If you want the deeper “trust boundary” explanation, use the Self-learn links at the top of the notebook.

In [ ]:
import os
from pathlib import Path
from typing import Optional


try:
    from dotenv import load_dotenv
except Exception as e:  # pragma: no cover
    load_dotenv = None
    _dotenv_import_error = e


def load_env(path: Optional[str] = None) -> None:
    if load_dotenv is None:
        raise RuntimeError("python-dotenv is required: %s" % _dotenv_import_error)
    if path:
        load_dotenv(dotenv_path=Path(path))
    else:
        load_dotenv()


def require_env(name: str) -> str:
    value = os.environ.get(name)
    if not value:
        raise RuntimeError("Missing %s. Put it in .env or set env var." % name)
    return value


print("env helpers ready")

In [ ]:
from typing import Any, Dict


def build_config() -> Dict[str, Any]:
    # TODO: build a config dict from env + defaults.
    # Example fields: model, timeout_s, max_retries.
    return {
        "model": os.environ.get("MODEL", "llama3.1"),
        "timeout_s": float(os.environ.get("TIMEOUT_S", "30")),
        "max_retries": int(os.environ.get("MAX_RETRIES", "3")),
    }


def redact_secret(secret: str, visible: int = 4) -> str:
    if len(secret) <= visible:
        return "*" * len(secret)
    return secret[:visible] + "*" * (len(secret) - visible)


print(redact_secret("sk-123456"))
print("config_preview:", build_config())
print("Implement build_config().")

## Common pitfalls

- putting `.env` in git by accident (fix: ensure `.gitignore` includes it)
- mixing config and secrets in code defaults (fix: read from env, fail with a clear message)
- unclear error messages when secrets are missing (fix: say exactly which variable is required)

## References

- Twelve-Factor config: https://12factor.net/config
- python-dotenv: https://github.com/theskumar/python-dotenv

## Appendix: Solutions (peek only after trying)

Reference implementations for `build_config` (with required secrets) and helpers.

In [ ]:
def build_config() -> Dict[str, Any]:
    # Example: require a key if you're calling a provider.
    api_key = os.environ.get("API_KEY")
    if api_key is None or not str(api_key).strip():
        raise RuntimeError("Missing API_KEY. Put it in .env or set env var.")

    return {
        "model": os.environ.get("MODEL", "llama3.1"),
        "timeout_s": float(os.environ.get("TIMEOUT_S", "30")),
        "max_retries": int(os.environ.get("MAX_RETRIES", "3")),
        "api_key_redacted": redact_secret(api_key),
    }


try:
    print("solution_config_preview:", build_config())
except RuntimeError as e:
    print("(expected in notebook env)", e)